In [1]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
import statsmodels.tools.tools as stattools
import jenkspy
from scipy import stats
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('E:\\term8\\datamining\\HW\\project\\prepared_data.csv')
breaks = jenkspy.jenks_breaks(df['age'],nb_class = 5)
df['agebin'] = pd.cut(x = df['age'], bins = breaks, right = False, labels = [1,2,3,4,5])

breaks = jenkspy.jenks_breaks(df['bmi'],nb_class = 5)
df['bmibin'] = pd.cut(x = df['bmi'], bins = breaks, right = False, labels = [1,2,3,4,5])

breaks = jenkspy.jenks_breaks(df['avg_glucose_level'],nb_class = 5)
df['glubin'] = pd.cut(x = df['avg_glucose_level'], bins = breaks, right = False, labels = [1,2,3,4,5])
df = df.drop(columns = ['age', 'bmi', 'avg_glucose_level'])
df = pd.get_dummies(df, columns = ['agebin', 'bmibin', 'glubin'])

df_train, df_test = train_test_split(df, test_size = 0.2, random_state = 45)

In [3]:
temp = df_train[df['stroke'] == 1]
temp = temp.sample(n = len(df_train[df_train['stroke'] == 0]) - len(df_train[df_train['stroke'] == 1]),
                   replace = True)#20% of data: 800
df_train = pd.concat([df_train, temp])
y = df_train['stroke'].values
df_train = df_train.drop(columns = ['stroke'])
X = df_train.values
y_test = df_test['stroke'].values
df_test = df_test.drop(columns = ['stroke'])
X_test = df_test.values
%time nb_01 = MultinomialNB().fit(X, y)
%time y_pred = nb_01.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 6.07 ms
Wall time: 13.5 ms
              precision    recall  f1-score   support

           0       0.97      0.71      0.82       782
           1       0.15      0.74      0.26        57

    accuracy                           0.71       839
   macro avg       0.56      0.72      0.54       839
weighted avg       0.92      0.71      0.78       839

[[552 230]
 [ 15  42]]


E:\term6\Anaconda\setUp\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [4]:
df = pd.read_csv('E:\\term8\\datamining\\HW\\project\\prepared_data.csv')
df_train, df_test = train_test_split(df, test_size = 0.2, random_state = 45) 
temp = df_train[df['stroke'] == 1]
temp = temp.sample(n = len(df_train[df_train['stroke'] == 0]) - len(df_train[df_train['stroke'] == 1]),
                   replace = True)#20% of data: 800
df_train = pd.concat([df_train, temp])
X = df_train.values[:, 0:20]
y = df_train.values[:, 20]
X_test = df_test.values[:, 0:20]
y_test = df_test.values[:, 20]
%time clf = LogisticRegression(random_state=0).fit(X, y)
%time y_pred = (clf.predict_proba(X_test)[:,1]>0.5)*1
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 39.5 ms
Wall time: 0 ns
              precision    recall  f1-score   support

         0.0       0.98      0.73      0.83       782
         1.0       0.17      0.75      0.27        57

    accuracy                           0.73       839
   macro avg       0.57      0.74      0.55       839
weighted avg       0.92      0.73      0.79       839

[[568 214]
 [ 14  43]]


E:\term6\Anaconda\setUp\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
%time clf = SVC(gamma='auto').fit(X, y)
%time y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 1.68 s
Wall time: 354 ms
              precision    recall  f1-score   support

         0.0       0.97      0.73      0.83       782
         1.0       0.16      0.70      0.26        57

    accuracy                           0.73       839
   macro avg       0.56      0.71      0.54       839
weighted avg       0.92      0.73      0.79       839

[[569 213]
 [ 17  40]]


In [6]:
mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=300,activation = 'relu', solver = 'adam', alpha=1)
%time mlp.fit(X, y)
%time y_pred = mlp.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 3.59 s
Wall time: 1.12 ms
              precision    recall  f1-score   support

         0.0       0.97      0.74      0.84       782
         1.0       0.17      0.72      0.28        57

    accuracy                           0.74       839
   macro avg       0.57      0.73      0.56       839
weighted avg       0.92      0.74      0.80       839

[[582 200]
 [ 16  41]]


In [7]:
DT = tree.DecisionTreeClassifier(criterion = 'gini',max_depth=4)
%time DT.fit(X,y)
%time y_pred = DT.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 10.7 ms
Wall time: 997 µs
              precision    recall  f1-score   support

         0.0       0.97      0.72      0.82       782
         1.0       0.14      0.65      0.24        57

    accuracy                           0.71       839
   macro avg       0.55      0.68      0.53       839
weighted avg       0.91      0.71      0.78       839

[[562 220]
 [ 20  37]]


In [8]:
DT = tree.DecisionTreeClassifier(criterion = 'entropy',max_depth=4)
%time DT.fit(X,y)
%time y_pred = DT.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 12.7 ms
Wall time: 0 ns
              precision    recall  f1-score   support

         0.0       0.97      0.69      0.80       782
         1.0       0.13      0.67      0.22        57

    accuracy                           0.69       839
   macro avg       0.55      0.68      0.51       839
weighted avg       0.91      0.69      0.76       839

[[537 245]
 [ 19  38]]


In [9]:
model = RandomForestClassifier(n_estimators=100,max_depth=4,
                               criterion='entropy', random_state=40)
%time model.fit(X,y)
%time y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 335 ms
Wall time: 13.9 ms
              precision    recall  f1-score   support

         0.0       0.98      0.66      0.79       782
         1.0       0.15      0.81      0.25        57

    accuracy                           0.67       839
   macro avg       0.56      0.74      0.52       839
weighted avg       0.92      0.67      0.75       839

[[519 263]
 [ 11  46]]


## subset of dataset (age under 75)

In [10]:
df = pd.read_csv('E:\\term8\\datamining\\HW\\project\\prepared_data_75.csv')
breaks = jenkspy.jenks_breaks(df['age'],nb_class = 5)
df['agebin'] = pd.cut(x = df['age'], bins = breaks, right = False, labels = [1,2,3,4,5])

breaks = jenkspy.jenks_breaks(df['bmi'],nb_class = 5)
df['bmibin'] = pd.cut(x = df['bmi'], bins = breaks, right = False, labels = [1,2,3,4,5])

breaks = jenkspy.jenks_breaks(df['avg_glucose_level'],nb_class = 5)
df['glubin'] = pd.cut(x = df['avg_glucose_level'], bins = breaks, right = False, labels = [1,2,3,4,5])
df = df.drop(columns = ['age', 'bmi', 'avg_glucose_level'])
df = pd.get_dummies(df, columns = ['agebin', 'bmibin', 'glubin'])

df_train, df_test = train_test_split(df, test_size = 0.2, random_state = 45)

In [11]:
temp = df_train[df['stroke'] == 1]
temp = temp.sample(n = len(df_train[df_train['stroke'] == 0]) - len(df_train[df_train['stroke'] == 1]),
                   replace = True)#20% of data: 800
df_train = pd.concat([df_train, temp])
y = df_train['stroke'].values
df_train = df_train.drop(columns = ['stroke'])
X = df_train.values
y_test = df_test['stroke'].values
df_test = df_test.drop(columns = ['stroke'])
X_test = df_test.values
%time nb_01 = MultinomialNB().fit(X, y)
%time y_pred = nb_01.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 3.33 ms
Wall time: 994 µs
              precision    recall  f1-score   support

           0       0.98      0.69      0.81       707
           1       0.09      0.72      0.15        29

    accuracy                           0.69       736
   macro avg       0.54      0.71      0.48       736
weighted avg       0.95      0.69      0.78       736

[[486 221]
 [  8  21]]


E:\term6\Anaconda\setUp\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [12]:
df = pd.read_csv('E:\\term8\\datamining\\HW\\project\\prepared_data_75.csv')
df_train, df_test = train_test_split(df, test_size = 0.2, random_state = 45) 
temp = df_train[df['stroke'] == 1]
temp = temp.sample(n = len(df_train[df_train['stroke'] == 0]) - len(df_train[df_train['stroke'] == 1]),
                   replace = True)#20% of data: 800
df_train = pd.concat([df_train, temp])
X = df_train.values[:, 0:20]
y = df_train.values[:, 20]
X_test = df_test.values[:, 0:20]
y_test = df_test.values[:, 20]
%time clf = LogisticRegression(random_state=0).fit(X, y)
%time y_pred = (clf.predict_proba(X_test)[:,1]>0.5)*1
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 38.1 ms
Wall time: 0 ns
              precision    recall  f1-score   support

         0.0       0.99      0.72      0.83       707
         1.0       0.11      0.83      0.19        29

    accuracy                           0.73       736
   macro avg       0.55      0.77      0.51       736
weighted avg       0.96      0.73      0.81       736

[[510 197]
 [  5  24]]


E:\term6\Anaconda\setUp\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
%time clf = SVC(gamma='auto').fit(X, y)
%time y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 1.33 s
Wall time: 373 ms
              precision    recall  f1-score   support

         0.0       0.97      0.76      0.86       707
         1.0       0.08      0.52      0.14        29

    accuracy                           0.75       736
   macro avg       0.53      0.64      0.50       736
weighted avg       0.94      0.75      0.83       736

[[540 167]
 [ 14  15]]


In [14]:
mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=300,activation = 'relu', solver = 'adam', alpha=1)
%time mlp.fit(X, y)
%time y_pred = mlp.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 3.31 s
Wall time: 6.96 ms
              precision    recall  f1-score   support

         0.0       0.98      0.78      0.87       707
         1.0       0.09      0.52      0.15        29

    accuracy                           0.77       736
   macro avg       0.53      0.65      0.51       736
weighted avg       0.94      0.77      0.84       736

[[554 153]
 [ 14  15]]


In [15]:
DT = tree.DecisionTreeClassifier(criterion = 'gini',max_depth=4)
%time DT.fit(X,y)
%time y_pred = DT.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 16.1 ms
Wall time: 1.18 ms
              precision    recall  f1-score   support

         0.0       0.98      0.77      0.86       707
         1.0       0.10      0.62      0.17        29

    accuracy                           0.76       736
   macro avg       0.54      0.70      0.52       736
weighted avg       0.95      0.76      0.83       736

[[544 163]
 [ 11  18]]


In [16]:
DT = tree.DecisionTreeClassifier(criterion = 'entropy',max_depth=4)
%time DT.fit(X,y)
%time y_pred = DT.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 15 ms
Wall time: 999 µs
              precision    recall  f1-score   support

         0.0       0.99      0.62      0.76       707
         1.0       0.08      0.79      0.14        29

    accuracy                           0.63       736
   macro avg       0.53      0.71      0.45       736
weighted avg       0.95      0.63      0.74       736

[[441 266]
 [  6  23]]


In [17]:
model = RandomForestClassifier(n_estimators=100,max_depth=4,
                               criterion='entropy', random_state=40)
%time model.fit(X,y)
%time y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels = [0, 1]))

Wall time: 313 ms
Wall time: 2.75 ms
              precision    recall  f1-score   support

         0.0       0.99      0.76      0.86       707
         1.0       0.11      0.72      0.19        29

    accuracy                           0.76       736
   macro avg       0.55      0.74      0.52       736
weighted avg       0.95      0.76      0.83       736

[[536 171]
 [  8  21]]
